# import module and setting parameters

In [2]:
import numpy as np
from math import e,log
from functools import reduce,partial
n,beta=200,0.5

# Introduction
In the present paper, we study semiparametric methods for analysing current status data under the additive hazards regression model. This model specifies that the hazard function of T at time t, given the history of a p-dimensional covariate process Z(.) up to t, has the form 
$$\lambda(t|\textbf(Z))=\lambda_{0}(t)+\beta_{0}^{'}Z(t)$$
where $\lambda_{0}(t)$ is an unspecified baseline hazard function, and f3o is a p-vector of unknown regression parameter

In inference procedures section,  assume that $C$ is independent of $T$ and $Z$. Let$\{T_{i},C_{i},Z_{i}\}(i=1,...,n)$  be independent replicates of $\{T_{i},C_{i},Z_{i}\}$The observations consist of $C_{i},\delta_{i}=I(C_{i}\leq T_{i})$ and $Z_{i}(t)(t \leq C_{i})$

# simulation
In paper , the failure times were generated from model:$$\lambda(t|\textbf(Z))=\lambda_{0}(t)+\beta_{0}^{'}Z(t)$$
with $\lambda_{0}(.)=1,\beta_{0}=0.5$,and $Z$ being uniform random variable on $(0,\sqrt{12})$

The monitoring times were generate from the exponential distribution with hazard rate $\lambda_{c,0},e^{\gamma_{0}Z}$, where
$\lambda_{c,0}=0.5,1.0,1.5$
## generate data
step1 generate data C from exponential(1) ,size=n

step2 generate data Z from uniform(0,$\sqrt{12}$) ,size=n

step3 $Z_{i}^{*}(t)=\int_{0}^{t}Z_{i}(s)ds$

step4 generate data T from exponentiao($\frac{1}{1+\beta*Z_{i}}$)

step5 calculate $\delta$ by $T_{i},C_{i}$

In [3]:
c=np.random.exponential(1,(n,1))
z=np.random.uniform(0,2*3**(1/2),(n,1))
T=list(map(lambda i:np.random.exponential(1/(1+beta*z[i]))[0],range(n)))
delta=np.array([1 if c[j,0]<=T[j] else 0 for j  in range(n)])
delta.shape=n,1
zz=c*z
cz=c.T*z

# Counting process matrix and Likelihood
Lilihood funciton is:
$$L(\beta)=\prod_{i=1}^{n}[\frac{e^{-\beta^{'}Z^{*}(t)}}{\sum Y_{j}(t)e^{-\beta^{'}Z^{*}(t)}}]^{\delta_{i}}$$
score function is:
$$U(\beta)=\sum_{i=1}^{n}(Z_{i}-\frac{\sum_{j=1}^{n}Y_{j}(C_{i})e^{\beta^{'}Z_{j}(C_{i})}Z_{j}(C_{i})}{\sum_{j=1}^{n}Y_{j}(C_{i})e^{\beta^{'}Z_{j}(C_{i})}})$$
Transform it into a matrix operation is:
$$\log(L(\beta))=\delta(-\beta^{'}Z^{*}-diag(Ye^{-\beta^{'}Z^{*}}))$$
$$U(\beta)=\delta (Z-diag(\frac{Ye^{-\beta^{'}Z^{*}}Z^{*}}{Ye^{-\beta^{'}Z^{*}}})$$

Then,by minimize$\log(L(\beta))$ or solve $U(\beta)=0$,we can also get the estimate of $\beta$

Before estimate $\beta$, we should first get the form for $Y,Z^{*}$

In [3]:
def eij(c):
    exp=np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            if c[j]>=c[i]:
                exp[i,j]=1
    return exp

def f(x,zz,exp,cz,n):
    f1=-x*zz
    f2=exp @ np.array(list(map(lambda i:e**i,-x*cz)))
    try:
        f3=f2 @ (-cz)@np.linalg.inv(f2)
    except:
        f3=np.zeros((n,n))
    z1,z2=f2.diagonal(),f3.diagonal()
    z1.shape=n,1
    z2.shape=n,1
    return f1-z1,zz-z2

Then I'll use Gradient descent to calculate the estimate of $\beta$


In [5]:
def simulation(n,beta,gamma,lambdac):
    z=np.random.uniform(0,12**(1/2),(n,1))
    c=list(map(lambda i:np.random.exponential(lambdac*e**(gamma*z[i]))[0],range(n)))
    T=list(map(lambda i:np.random.exponential(1/(1+beta*z[i]))[0],range(n)))
    delta=np.array([1 if c[j]<=T[j] else 0 for j  in range(n)])
    delta.shape=n,1
    exp=eij(c)
    
    zz=np.array([round(c[i]*z[i,0],4) for i in range(n)])
    zz.shape=n,1
    cz=c*z
    
    beta0,k=np.random.uniform(0,1),1
    fn,grad=-delta.T@f(beta0,zz,exp,cz,n)
    while np.abs(fn/grad)>=0.001 and k<=1000:
        beta0 -=fn/grad
        fn,grad=-delta.T@f(beta0,zz,exp,cz,n)
        k+=1
    return beta0
